# Lucas-Kanade Analysis

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

evaluation_dir = utils.dir_path('outputs', 'signals')

predictions_file = utils.join_paths(evaluation_dir, 'lucas_kanade.csv')
predictions = pd.read_csv(predictions_file)
predictions['signal'] = predictions['signal'].apply(eval).apply(np.array)
predictions['duration'] = pd.to_timedelta(predictions['duration'])
predictions.head()

## Compare the predictions to the ground truth

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

In [ ]:
import respiration.analysis as analysis

methods = predictions['method'].unique()
use_cgof = [True, False]
grey = [True, False]

analysis_results = []

for method in methods:
    for g in grey:
        signals = predictions[
            (predictions['method'] == method) &
            (predictions['grey'] == g)]

        experiment_analysis = analysis.Analysis()

        for idx, row in signals.iterrows():
            subject, setting = row['subject'], row['setting']
            prediction = row['signal']
            gt_signal = dataset.get_breathing_signal(subject, setting)
            experiment_analysis.add_data(prediction, gt_signal, 30)

        metrics = experiment_analysis.compute_metrics()
        for entry in metrics:
            analysis_results.append({
                'model': method,
                'grey': g,
                'metric': entry['metric'],
                'method': entry['method'],
                'value': entry['value'],
            })

analysis_results = pd.DataFrame(analysis_results)

In [ ]:
# Only shoe MAE and pk
analysis_results[
    (analysis_results['metric'] == 'Correlation')
    # (analysis_results['method'] == 'pk')
    ]